In [1]:
!pip install ortools

'pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [2]:
from ortools.linear_solver import pywraplp

In [3]:
# declara o solver: se PLI, usa-se SCIP; se PL, usa-se GLOP.
solver = pywraplp.Solver.CreateSolver('SCIP')

# declara um número muito grande caso seja necessário usar no modelo
infinity = solver.infinity()

In [4]:
def ler_dados_entrada(arquivo):
    """


    Formato esperado do arquivo:
    Linha 1: tamanho da barra original
    Linha 2: quantidade de tipos de itens
    Linha 3: tamanhos dos itens (separados por espaço)
    Linha 4: demandas dos itens (separados por espaço)

    """
    with open(arquivo, 'r') as f:
        linhas = f.readlines()

    tamanho_barra = int(linhas[0].strip())
    n_tipos = int(linhas[1].strip())
    tamanhos = list(map(int, linhas[2].strip().split()))
    demandas = list(map(int, linhas[3].strip().split()))

    return tamanho_barra, tamanhos, demandas


arquivo_entrada = 'entrada.txt'


tamanho_barra, tamanhos, demandas = ler_dados_entrada(arquivo_entrada)
print(f"Dados lidos do arquivo '{arquivo_entrada}'")

print(f"\nTamanho da barra original: {tamanho_barra}m")
print(f"Tipos de itens: {len(tamanhos)}")
for i, (tam, dem) in enumerate(zip(tamanhos, demandas)):
    print(f"  Item {i+1}: {dem} barras de {tam}m")

Dados lidos do arquivo 'entrada.txt'

Tamanho da barra original: 150m
Tipos de itens: 3
  Item 1: 70 barras de 80m
  Item 2: 100 barras de 60m
  Item 3: 120 barras de 50m


## Geração de Padrões de Corte


In [5]:
def gerar_padroes_corte_maximal(tamanho_barra, tamanhos):

    n_itens = len(tamanhos)
    padroes = []
    def gerar_recursivo(indice, espaco_restante, padrao_atual):
        if indice == n_itens:

            pode_melhorar = False
            for k in range(n_itens):
                if espaco_restante >= tamanhos[k]:
                    pode_melhorar = True
                    break
            if not pode_melhorar and sum(padrao_atual) > 0:
                padroes.append(padrao_atual[:])
            return
        max_quantidade = espaco_restante // tamanhos[indice]
        for quantidade in range(max_quantidade + 1):
            padrao_atual[indice] = quantidade
            novo_espaco = espaco_restante - (quantidade * tamanhos[indice])
            gerar_recursivo(indice + 1, novo_espaco, padrao_atual)
    padrao_inicial = [0] * n_itens
    gerar_recursivo(0, tamanho_barra, padrao_inicial)
    return padroes

In [6]:
# Gera apenas os padrões de corte maximal
padroes = gerar_padroes_corte_maximal(tamanho_barra, tamanhos)

padroes = padroes[::-1]  # Inverte a lista para melhor visualização

print(f"Total de padrões de corte gerados: {len(padroes)}\n")
print("Padrões de corte:")
print("-" * 80)

for i, padrao in enumerate(padroes):
    material_usado = sum(padrao[j] * tamanhos[j] for j in range(len(tamanhos)))
    desperdicio = tamanho_barra - material_usado
    # descrição do padrão como soma dos tamanhos
    descricao = []
    for j in range(len(tamanhos)):
        descricao.extend([str(tamanhos[j])] * padrao[j])
    descricao_str = " + ".join(descricao)
    print(f"p{i+1}: {descricao_str} = {material_usado}m usados (desperdício: {desperdicio}m)")
    print(f"     {padrao}")

Total de padrões de corte gerados: 5

Padrões de corte:
--------------------------------------------------------------------------------
p1: 80 + 60 = 140m usados (desperdício: 10m)
     [1, 1, 0]
p2: 80 + 50 = 130m usados (desperdício: 20m)
     [1, 0, 1]
p3: 60 + 60 = 120m usados (desperdício: 30m)
     [0, 2, 0]
p4: 60 + 50 = 110m usados (desperdício: 40m)
     [0, 1, 1]
p5: 50 + 50 + 50 = 150m usados (desperdício: 0m)
     [0, 0, 3]


In [7]:
#demandas de cada corte
demands_indexes = [[0] * len(padroes) for _ in range(len(tamanhos))]
for x in range(len(tamanhos)):
    for i in range(len(padroes)):
        demands_indexes[x][i] = padroes[i][x]
print(demands_indexes)

[[1, 1, 0, 0, 0], [1, 0, 2, 1, 0], [0, 1, 0, 1, 3]]


In [8]:
#desperdicios
waste = [0]*len(padroes)
for x in range(len(padroes)):
    cut_total = 0
    for i in range(len(tamanhos)):
        cut_total += tamanhos[i]*padroes[x][i]
    waste[x] = tamanho_barra - cut_total
print(waste)

[10, 20, 30, 40, 0]


In [9]:
x = {}
for i in range(len(padroes)):
    x[i] = solver.IntVar(0, infinity, 'x' + str(i+1))

In [10]:
for i in range(len(demandas)):
  restricao = solver.RowConstraint(demandas[i], infinity, '-')
  for j in range(len(padroes)):
    restricao.SetCoefficient(x[j], demands_indexes[i][j])

In [11]:
# declara a função objetivo
objetivo = solver.Objective()
for i in range(len(padroes)):
  objetivo.SetCoefficient(x[i], waste[i])
objetivo.SetMinimization()

In [12]:
print(solver.ExportModelAsLpFormat(False))

\ Generated by MPModelProtoExporter
\   Name             : 
\   Format           : Free
\   Constraints      : 3
\   Variables        : 5
\     Binary         : 0
\     Integer        : 5
\     Continuous     : 0
Minimize
 Obj: +10 x1 +20 x2 +30 x3 +40 x4 
Subject to
 _: +1 x1 +1 x2  >= 70
 __1: +1 x1 +2 x3 +1 x4  >= 100
 __2: +1 x2 +1 x4 +3 x5  >= 120
Bounds
 0 <= x1 <= inf
 0 <= x2 <= inf
 0 <= x3 <= inf
 0 <= x4 <= inf
 0 <= x5 <= inf
Generals
 x1
 x2
 x3
 x4
 x5
End



In [13]:
# resolve o modelo
status = solver.Solve()

In [14]:
# verifica se a solução é ótima e, caso seja, exibe ela
if status == pywraplp.Solver.OPTIMAL:
  print('gasto:', objetivo.Value())
  for i in range(len(padroes)):
    print(x[i].name(), x[i].solution_value())
else:
  print('Modelo sem solução ótima.')

gasto: 1000.0
x1 100.0
x2 0.0
x3 0.0
x4 0.0
x5 40.0
